In [1]:
import numpy as np
from finite_volume.advection import AdvectionSolver
from finite_volume.initial_conditions import generate_ic

def l1(x: np.ndarray) -> float:
    return np.mean(np.abs(x))


def l2(x: np.ndarray) -> float:
    return np.sqrt(np.mean(np.square(x)))


def linf(x: np.ndarray) -> float:
    return np.max(np.abs(x))

In [2]:
p=1
a=1
k=-2
quadrature='gauss-legendre'
ic_type__PAD="square",(0,1)
limiter_config=dict(
    aposteriori_limiting=True, 
    SED=True, 
    hancock=True, 
    convex=False, 
    fallback_to_first_order=False, 
    fallback_limiter='minmod'
    )

In [3]:
ic_type, PAD = ic_type__PAD

def linear_transformation(x):
    return a * x + k

def u0(x, y):
    return generate_ic(type=ic_type, x=x, y=y)

def u0_shifted(x, y):
    return linear_transformation(u0(x, y))

u0_shifted.__name__ += f"_{a}_{k}"

shared_config = dict(
    **limiter_config,
    save=False,
    load=False,
    v=(2, 1),
    n=(64,),
    order=p + 1,
    courant=0.8,
    snapshot_dt=1,
    flux_strategy=quadrature,
)

# baseline
solver = AdvectionSolver(
    **shared_config,
    u0=u0,
    PAD=PAD,
)
# solver.one_euler_step(n=5)
solver.rkorder()

# shifted initial condition
translated_solver = AdvectionSolver(
    **shared_config,
    u0=u0_shifted,
    PAD=sorted((linear_transformation(PAD[0]), linear_transformation(PAD[1]))),
)
# translated_solver.one_euler_step(n=5)
translated_solver.rkorder()

# check equivariance
diffs = (
    linear_transformation(solver.u_snapshots[-1][1])
    - translated_solver.u_snapshots[-1][1]
)
print(f"{l1(diffs)=}")
print(f"{l2(diffs)=}")
print(f"{linf(diffs)=}")

New solution instance...


  0%|          | 0/1 [00:00]

100%|██████████| 1.0/1 [00:03]                 



New solution instance...


100%|██████████| 1.0/1 [00:02]                 


l1(diffs)=2.857414825585547e-16
l2(diffs)=4.2402546783265376e-16
linf(diffs)=2.220446049250313e-15
